In [1]:
!pip install azure-storage-blob # Microoft Azure
!pip install pyarrow
!pip install psycopg2 sqlalchemy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 392.2/392.2 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.1 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import json
import requests
from io import StringIO
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from math import ceil
import datetime
import calendar
from sqlalchemy import create_engine

In [3]:
# Azure Functions
def azure_upload_blob(connect_str, container_name, blob_name, data):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    blob_client.upload_blob(data, overwrite=True)
    print(f"Uploaded to Azure Blob: {blob_name}")

def azure_download_blob(connect_str, container_name, blob_name):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    download_stream = blob_client.download_blob()
    return download_stream.readall()

# Google Cloud Functions
def google_upload_blob(bucket_name, source_file_name, destination_blob_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)
    print(f"File {source_file_name} uploaded to {destination_blob_name}.")

def google_download_blob(bucket_name, source_blob_name, destination_file_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(source_blob_name)
    blob.download_to_filename(destination_file_name)
    print(f"Blob {source_blob_name} downloaded to {destination_file_name}.")

# AWS Functions
def aws_upload_file(file_name, bucket, object_name=None):
    if object_name is None:
        object_name = os.path.basename(file_name)
    s3_client = boto3.client('s3')
    response = s3_client.upload_file(file_name, bucket, object_name)
    print(f"Uploaded {file_name} to S3 bucket {bucket}.")

def aws_download_file(bucket, object_name, file_name):
    s3_client = boto3.client('s3')
    s3_client.download_file(bucket, object_name, file_name)
    print(f"Downloaded {object_name} from S3 bucket {bucket}.")

In [8]:
import pandas as pd

# Load data directly from the NYC Open Data API
url = "https://data.cityofnewyork.us/resource/w7w3-xahh.csv"
data = pd.read_csv(url)

# Basic data cleaning
data['business_name'] = data['business_name'].str.upper()
data['license_creation_date'] = pd.to_datetime(data['license_creation_date'])

# Remove duplicates
data = data.drop_duplicates()

# Fill missing values


# Data transformation example: Extract year from date


In [9]:
# Display basic information and the first few rows to understand what the data looks like
print(data.info())
print(data.head())

# Check for the number of missing values per column
print(data.isnull().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   license_nbr            1000 non-null   object        
 1   license_type           1000 non-null   object        
 2   lic_expir_dd           984 non-null    object        
 3   license_status         1000 non-null   object        
 4   license_creation_date  1000 non-null   datetime64[ns]
 5   industry               1000 non-null   object        
 6   business_name          997 non-null    object        
 7   business_name_2        192 non-null    object        
 8   address_building       756 non-null    object        
 9   address_street_name    759 non-null    object        
 10  address_street_name_2  44 non-null     object        
 11  address_city           998 non-null    object        
 12  address_state          997 non-null    object        
 13  addr

In [11]:
# Filling missing values with a placeholder or the most common value
data['business_name'].fillna('Unknown Business', inplace=True)
data['address_city'].fillna('Unknown City', inplace=True)
data['address_state'].fillna(data['address_state'].mode()[0], inplace=True)  # mode for categorical
data['address_zip'].fillna('Unknown ZIP', inplace=True)


In [12]:
# Dropping columns with excessive missing values
data.drop(columns=['business_name_2', 'address_street_name_2', 'detail_2'], inplace=True)


In [14]:
# Filling geographical data with a default or mean value
default_longitude = data['longitude'].mean()
default_latitude = data['latitude'].mean()
data['longitude'].fillna(default_longitude, inplace=True)
data['latitude'].fillna(default_latitude, inplace=True)

# For contact phone, consider a placeholder
data['contact_phone'].fillna('No Contact Available', inplace=True)



In [15]:
# Conditional filling based on other data or dropping
data['address_building'].fillna('No Building Info', inplace=True)
data['address_borough'].fillna('Unknown Borough', inplace=True)


In [16]:
# Convert date fields to datetime objects
data['license_creation_date'] = pd.to_datetime(data['license_creation_date'])
data['lic_expir_dd'] = pd.to_datetime(data['lic_expir_dd'], errors='coerce')  # Handle errors if format varies

# Categorical data
data['license_type'] = data['license_type'].astype('category')
data['industry'] = data['industry'].astype('category')
data['license_status'] = data['license_status'].astype('category')


In [17]:
data.drop_duplicates(subset=['license_nbr', 'business_name', 'address_zip'], inplace=True)


In [18]:
data.to_csv('cleaned_legally_operating_businesses.csv', index=False)


In [19]:
import pandas as pd

# Assuming data has already been cleaned and loaded into a DataFrame named 'data'
data = pd.read_csv('cleaned_legally_operating_businesses.csv')  # Example loading cleaned data

# Create a DataFrame to hold data mapping information
data_mapping = pd.DataFrame({
    'Field Name': data.columns,
    'Data Type': data.dtypes.astype(str),
    'Description': ['Description of ' + col for col in data.columns],  # Placeholder descriptions
    'Source Column': data.columns,  # Assuming direct mapping for simplicity
    'Destination Column': data.columns  # Assuming no change in column names after cleaning
})

# Display the data mapping
print(data_mapping)

                                  Field Name Data Type  \
license_nbr                      license_nbr    object   
license_type                    license_type    object   
lic_expir_dd                    lic_expir_dd    object   
license_status                license_status    object   
license_creation_date  license_creation_date    object   
industry                            industry    object   
business_name                  business_name    object   
address_building            address_building    object   
address_street_name      address_street_name    object   
address_city                    address_city    object   
address_state                  address_state    object   
address_zip                      address_zip    object   
contact_phone                  contact_phone    object   
address_borough              address_borough    object   
detail                                detail   float64   
community_board              community_board   float64   
council_distri

In [20]:
# Export to CSV
data_mapping.to_csv('data_mapping.csv', index=False)

# Optionally, integrate into documentation tools or data dictionaries as needed

In [21]:
descriptions = {
    'license_nbr': "Unique identifier for each business license issued.",
    'license_type': "Categorization of the license according to the regulatory standards.",
    'lic_expir_dd': "The expiration date of the business license.",
    'license_status': "Current status of the license (e.g., Active, Expired, Suspended).",
    'license_creation_date': "The date on which the license was initially issued.",
    'industry': "The sector or category of business activity as defined by regulatory guidelines.",
    'business_name': "Official registered name of the business entity.",
    'address_city': "City in which the business is located.",
    'address_state': "State in which the business is located.",
    'address_zip': "ZIP code for the business's location.",
    'contact_phone': "Contact phone number for the business."
}

# Update the data mapping DataFrame with these descriptions
data_mapping['Description'] = data_mapping['Field Name'].map(descriptions)


In [22]:
import pandas as pd

# Assuming the cleaned data was saved as 'cleaned_legally_operating_businesses.csv'
data = pd.read_csv('cleaned_legally_operating_businesses.csv')


In [23]:
# Standardize date format and extract components
data['license_creation_date'] = pd.to_datetime(data['license_creation_date'], errors='coerce')
data['Year'] = data['license_creation_date'].dt.year
data['Month'] = data['license_creation_date'].dt.month
data['Day'] = data['license_creation_date'].dt.day


In [24]:
# One-hot encoding for 'industry' column
industry_dummies = pd.get_dummies(data['industry'], prefix='industry')
data = pd.concat([data, industry_dummies], axis=1)


In [25]:
# Assuming 'lic_expir_dd' is also a date column
data['lic_expir_dd'] = pd.to_datetime(data['lic_expir_dd'], errors='coerce')
data['Days_Until_Expiry'] = (data['lic_expir_dd'] - pd.to_datetime('today')).dt.days


In [26]:
# Standardizing text data
data['business_name'] = data['business_name'].str.title()  # Capitalize first letter of each word


In [27]:
# Drop original 'industry' column after encoding
data.drop('industry', axis=1, inplace=True)


In [28]:
from sklearn.preprocessing import MinMaxScaler

# Normalize a numeric column, e.g., 'Days_Until_Expiry'
scaler = MinMaxScaler()
data['Days_Until_Expiry_Normalized'] = scaler.fit_transform(data[['Days_Until_Expiry']])


In [29]:
# Check the final DataFrame to ensure all transformations are applied correctly
print(data.head())

# Save the transformed data back to CSV
data.to_csv('transformed_legally_operating_businesses.csv', index=False)


   license_nbr license_type lic_expir_dd license_status license_creation_date  \
0  0967332-DCA     Business   2017-02-28       Inactive            2010-02-04   
1  2103411-DCA   Individual   2024-03-31         Active            2022-01-11   
2  2060087-DCA   Individual   2024-03-31         Active            2017-10-31   
3  2078026-DCA   Individual   2024-03-31         Active            2018-09-11   
4  2057801-DCA   Individual   2019-02-28       Inactive            2017-09-01   

         business_name  address_building address_street_name address_city  \
0  Barbarino, John Jr.               239          MEDFORD CT    MANALAPAN   
1      Nieciak, Joseph  No Building Info                 NaN     BROOKLYN   
2        Arcaro, Tyler  No Building Info                 NaN     NEW YORK   
3       Hesse, Daphnee  No Building Info                 NaN    Vancouver   
4       Escobar, Kevin  No Building Info                 NaN      CHESTER   

      address_state  ... industry_Special Sale ind